In [1]:
# Check for GPU specs
!nvidia-smi

Tue Jun 14 01:02:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installing parlAI

In [2]:
!pip install -q parlai
!pip install -q subword_nmt # extra requirement we need for this tutorial

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 208 kB 72.9 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 2.7 MB 60.2 MB/s 
     |████████████████████████████████| 8.9 MB 48.4 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 48 kB 5.5 MB/s 
     |████████████████████████████████| 132 kB 75.3 MB/s 
     |████████████████████████████████| 2.8 MB 57.6 MB/s 
     |████████████████████████████████| 248 kB 74.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 15.7 MB 26.3 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
     |████████████████████████████████| 181 kB 76.1 MB/s 
     |█████████████████████

# Import packages

In [3]:
# Import the Interactive script
from parlai.scripts.interactive import Interactive

# The display_data script is used to show the contents of a particular task.
# By default, shows train set
from parlai.scripts.display_data import DisplayData

# Script for training model
from parlai.scripts.train_model import TrainModel

# Visualize model prediction
from parlai.scripts.display_model import DisplayModel


In [4]:
!git clone https://github.com/nbertagnolli/counsel-chat.git

Cloning into 'counsel-chat'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 11), reused 25 (delta 4), pack-reused 0
Unpacking objects: 100% (35/35), done.


# Data selection

In [48]:
DisplayData.main(task='personachat', num_examples=20)

19:21:43 | Opt:
19:21:43 |     allow_missing_init_opts: False
19:21:43 |     batchsize: 1
19:21:43 |     datapath: ./data
19:21:43 |     datatype: train:ordered
19:21:43 |     dict_class: None
19:21:43 |     display_add_fields: 
19:21:43 |     download_path: None
19:21:43 |     dynamic_batching: None
19:21:43 |     hide_labels: False
19:21:43 |     ignore_agent_reply: True
19:21:43 |     image_cropsize: 224
19:21:43 |     image_mode: raw
19:21:43 |     image_size: 256
19:21:43 |     init_model: None
19:21:43 |     init_opt: None
19:21:43 |     is_debug: False
19:21:43 |     loglevel: info
19:21:43 |     max_display_len: 1000
19:21:43 |     model: None
19:21:43 |     model_file: None
19:21:43 |     multitask_weights: [1]
19:21:43 |     mutators: None
19:21:43 |     num_examples: 20
19:21:43 |     override: "{'task': 'personachat', 'num_examples': 20}"
19:21:43 |     parlai_home: /usr/local/lib/python3.7/dist-packages
19:21:43 |     starttime: Jun13_19-21
19:21:43 |     task: personachat

# Model selection

In [5]:
# note that if you want to see model-specific arguments, you must specify a model name
print(TrainModel.help(model='transformer/generator'))

usage: TrainModel [-h] [--helpall] [-o INIT_OPT]
                  [--allow-missing-init-opts ALLOW_MISSING_INIT_OPTS]
                  [-t TASK] [-dt DATATYPE] [-bs BATCHSIZE]
                  [-dynb {batchsort,None,full}] [-v] [--debug] [-dp DATAPATH]
                  [-m MODEL] [-mf MODEL_FILE] [-im INIT_MODEL] [-et EVALTASK]
                  [--final-extra-opt FINAL_EXTRA_OPT]
                  [--eval-dynamic-batching {batchsort,off,None,full}]
                  [--num-workers NUM_WORKERS] [-eps NUM_EPOCHS]
                  [-ttim MAX_TRAIN_TIME] [-tstep MAX_TRAIN_STEPS]
                  [-lstep LOG_EVERY_N_STEPS] [-vtim VALIDATION_EVERY_N_SECS]
                  [-vstep VALIDATION_EVERY_N_STEPS] [-stim SAVE_EVERY_N_SECS]
                  [-sval SAVE_AFTER_VALID] [-veps VALIDATION_EVERY_N_EPOCHS]
                  [-vp VALIDATION_PATIENCE] [-vmt VALIDATION_METRIC]
                  [-vmm {max,min}] [-mcs METRICS] [-micro AGGREGATE_MICRO]
                  [--world-logs WORL

# Training a model

## Training from scratch

In [6]:
!mkdir ./data

In [18]:
# we'll save it in the "from_scratch_model" directory
!rm -rf from_scratch_model
!mkdir -p from_scratch_model

In [19]:
from parlai.scripts.train_model import TrainModel
TrainModel.main(
    
    # Main parlai arguments
    task='empathetic_dialogues',  # train on blended_skill_talk
    datapath = './data',

    # Model arguments
    model='transformer/generator',     # we specify the model type as seq2seq
    model_file='from_scratch_model/model', # we MUST provide a filename
   
    # Training loop arguments
    max_train_time=600,
    
    # Seq2Seq arguments/hyperparameters
    #hiddensize = 128,


    # Torchagent argument
    embedding_type = 'fasttext',

)

15:56:07 | building dictionary first...
15:56:07 | Opt:
15:56:07 |     activation: relu
15:56:07 |     adafactor_eps: '(1e-30, 0.001)'
15:56:07 |     adam_eps: 1e-08
15:56:07 |     add_p1_after_newln: False
15:56:07 |     aggregate_micro: False
15:56:07 |     allow_missing_init_opts: False
15:56:07 |     attention_dropout: 0.0
15:56:07 |     batchsize: 1
15:56:07 |     beam_block_full_context: True
15:56:07 |     beam_block_list_filename: None
15:56:07 |     beam_block_ngram: -1
15:56:07 |     beam_context_block_ngram: -1
15:56:07 |     beam_delay: 30
15:56:07 |     beam_length_penalty: 0.65
15:56:07 |     beam_min_length: 1
15:56:07 |     beam_size: 1
15:56:07 |     betas: '(0.9, 0.999)'
15:56:07 |     bpe_add_prefix_space: None
15:56:07 |     bpe_debug: False
15:56:07 |     bpe_dropout: None
15:56:07 |     bpe_merge: None
15:56:07 |     bpe_vocab: None
15:56:07 |     checkpoint_activations: False
15:56:07 |     compute_tokenized_bleu: False
15:56:07 |     datapath: ./data
15:56:07 | 

Building dictionary: 100%|██████████| 64.6k/64.6k [00:03<00:00, 18.1kex/s]

15:56:14 | Saving dictionary to from_scratch_model/model.dict


15:56:14 | dictionary built with 22419 tokens in 0.0s
15:56:14 | No model with opt yet at: from_scratch_model/model(.opt)
15:56:14 | Using CUDA
15:56:14 | loading dictionary from from_scratch_model/model.dict
15:56:14 | num words = 22419


./data/models/fasttext_vectors/wiki.en.vec: 6.60GB [01:46, 62.2MB/s]                            
100%|██████████| 2519370/2519370 [03:50<00:00, 10924.02it/s]


16:02:49 | Initialized embeddings for 16270 tokens (72.6%) from fasttext.
16:02:49 | Total parameters: 10,235,700 (9,621,300 trainable)
16:02:49 | Opt:
16:02:49 |     activation: relu
16:02:49 |     adafactor_eps: '(1e-30, 0.001)'
16:02:49 |     adam_eps: 1e-08
16:02:49 |     add_p1_after_newln: False
16:02:49 |     aggregate_micro: False
16:02:49 |     allow_missing_init_opts: False
16:02:49 |     attention_dropout: 0.0
16:02:49 |     batchsize: 1
16:02:49 |     beam_block_full_context: True
16:02:49 |     beam_block_list_filename: None
16:02:49 |     beam_block_ngram: -1
16:02:49 |     beam_context_block_ngram: -1
16:02:49 |     beam_delay: 30
16:02:49 |     beam_length_penalty: 0.65
16:02:49 |     beam_min_length: 1
16:02:49 |     beam_size: 1
16:02:49 |     betas: '(0.9, 0.999)'
16:02:49 |     bpe_add_prefix_space: None
16:02:49 |     bpe_debug: False
16:02:49 |     bpe_dropout: None
16:02:49 |     bpe_merge: None
16:02:49 |     bpe_vocab: None
16:02:49 |     checkpoint_activations

({'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.002013),
  'clen': AverageMetric(39.52),
  'ctpb': GlobalAverageMetric(39.52),
  'ctps': GlobalTimerMetric(202),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'exps': GlobalTimerMetric(5.112),
  'exs': SumMetric(5738),
  'f1': F1Metric(0.1195),
  'gen_n_toks': AverageMetric(60.81),
  'gpu_mem': GlobalAverageMetric(0.003455),
  'llen': AverageMetric(15.65),
  'loss': AverageMetric(4.107),
  'lr': GlobalAverageMetric(1),
  'ltpb': GlobalAverageMetric(15.65),
  'ltps': GlobalTimerMetric(80.01),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'ppl': PPLMetric(60.77),
  'token_acc': AverageMetric(0.2554),
  'token_em': AverageMetric(0),
  'total_train_updates': GlobalFixedMetric(3.348e+04),
  'tpb': GlobalAverageMetric(55.17),
  'tps': GlobalTimerMetric(282)},
 {'accuracy': ExactMatchMetric(0),
  'bleu-4': BleuMetric(0.002045),
  'clen': AverageMetric(42.71),
  'ctpb': GlobalAverageMetric(42.71),

In [21]:
!zip -r ./ED_transformer.zip ./from_scratch_model

  adding: from_scratch_model/ (stored 0%)
  adding: from_scratch_model/model.valid (deflated 46%)
  adding: from_scratch_model/model.trainstats (deflated 82%)
  adding: from_scratch_model/model.test (deflated 46%)
  adding: from_scratch_model/model (deflated 8%)
  adding: from_scratch_model/model.dict.opt (deflated 67%)
  adding: from_scratch_model/model.dict (deflated 59%)
  adding: from_scratch_model/model.opt (deflated 66%)


## Fine-tuning

In [ ]:
#zoo:blended_skill_talk/multi_task_bst_tuned/model

In [13]:
!rm -rf fine_tune_personachat
!mkdir -p fine_tune_personachat

In [40]:
!rm -rf from_pretrained
!mkdir -p from_pretrained

In [41]:
# Improvement using transfer learning
TrainModel.main(
    # similar to before
    task='personachat', 
    datapath='./data',

    # model argument
    model='transformer/polyencoder',
    model_file='from_pretrained/model',

    max_train_time=60*3, 
    batchsize=64, eval_batchsize=10,
    warmup_updates=100, lr_scheduler_patience=0,
    lr_scheduler_decay=0.4,

    lr=5e-05, data_parallel=True, history_size=20, 
    
    candidates='batch',
    eval_candidates='batch',
    optimizer='adamax',
    output_scaling=0.06,

    variant='xlm',
    reduction_type='mean',
    share_encoders=False,

    attention_dropout=0.1,
    relu_dropout=0.0,
    dropout=0.1,

    activation='gelu',
    polyencoder_type='n_first',


    
    # initialize with a pretrained model
    init_model='zoo:pretrained_transformers/poly_model_huge_reddit/model', # which pretrained model you want to use
    dict_file = 'zoo:pretrained_transformers/poly_model_huge_reddit/model.dict', # where to find the pretrained model dictionary


    label_truncate=72,
    text_truncate=360,
    fp16=True,
    dict_tokenizer='bpe',
    dict_lower=True,

    validation_metric='accuracy',
    validation_metric_mode='max',
  
    learn_positional_embeddings=True,
    n_layers=12, n_heads=12, ffn_size=3072,
    embedding_size=768,
    n_positions=1024,
    learn_embeddings=True,
    embeddings_scale=False,
    n_segments=2,
    poly_n_codes=64,
    poly_attention_type='basic',
    dict_endtoken='__start__'

)

17:59:52 | building dictionary first...
17:59:52 | No model with opt yet at: from_pretrained/model(.opt)
17:59:52 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: None,verbose: False,is_debug: False,datapath: ./data,evaltask: None,final_extra_opt: ,eval_batchsize: 10,eval_dynamic_batching: None,num_workers: 0,display_examples: False,num_epochs: -1,max_train_time: 180.0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_secs: -1,validation_every_n_steps: -1,save_every_n_secs: -1,save_after_valid: False,validation_every_n_epochs: -1,validation_max_exs: -1,short_final_eval: False,validation_patience: 10,validation_metric: accuracy,validation_metric_mode: max,validation_cutoff: 1.0,load_from_checkpoint: True,validation_share_agent: False,metrics: default,aggregate_micro: False,world_logs: ,save_format: conversations,log_keep_fields: all,t

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


17:59:58 |     image_mode: raw
17:59:58 |     image_size: 256
17:59:58 |     inference: max
17:59:58 |     init_model: ./data/models/pretrained_transformers/poly_model_huge_reddit/model
17:59:58 |     init_opt: None
17:59:58 |     interactive_candidates: fixed
17:59:58 |     interactive_mode: False
17:59:58 |     invsqrt_lr_decay_gamma: -1
17:59:58 |     is_debug: False
17:59:58 |     label_truncate: 72
17:59:58 |     learn_embeddings: True
17:59:58 |     learn_positional_embeddings: True
17:59:58 |     learningrate: 5e-05
17:59:58 |     load_from_checkpoint: True
17:59:58 |     log_every_n_secs: -1
17:59:58 |     log_every_n_steps: 50
17:59:58 |     log_keep_fields: all
17:59:58 |     loglevel: info
17:59:58 |     lr_scheduler: reduceonplateau
17:59:58 |     lr_scheduler_decay: 0.4
17:59:58 |     lr_scheduler_patience: 0
17:59:58 |     max_train_steps: -1
17:59:58 |     max_train_time: 180.0
17:59:58 |     memory_attention: sqrt
17:59:58 |     metrics: default
17:59:58 |     model: tr

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


18:00:02 | training...
18:00:02 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:02 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:03 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:04 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:05 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:05 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:06 | Ran out of memory, skipping batch. if this happens frequently, decrease batchsize or truncate the inputs to the model.
18:00:06 | Ran out of memory, skipping batch. if this happens frequ

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


18:03:10 | [ Executing eval mode with batch labels as set of candidates. ]
18:04:36 | [ Warning: using candidate source 'batch' and observed a batch of size 1. This may be due to uneven batch sizes at the end of an epoch. ]
18:04:37 | eval completed in 86.38s
18:04:37 | valid:
    accuracy  bleu-4  clen  ctpb  ctps  ctrunc  ctrunclen  exps  exs    f1  gpu_mem  hits@1  hits@10  hits@100  hits@5  llen  \
       .2307   .2308 155.8  1570 14254       0          0 90.34 7801 .3182    .4421   .2307        1         1   .7214 14.35   
    loss    lr  ltpb  ltps  ltrunc  ltrunclen   mrr  rank  tpb   tps  
   4.727 5e-09 142.8  1297       0          0 .4398  3.95 1713 15551

18:04:37 | creating task(s): personachat
18:04:37 | loading fbdialog data: ./data/Persona-Chat/personachat/test_self_original.txt
18:04:37 | running eval: test


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


18:05:58 | eval completed in 81.36s
18:05:58 | test:
    accuracy  bleu-4  clen  ctpb  ctps  ctrunc  ctrunclen  exps  exs    f1  gpu_mem  hits@1  hits@10  hits@100  hits@5  llen  \
       .1982   .1988 152.5  1535 14271       0          0 92.36 7512 .2929    .4422   .1982        1         1   .7049 14.24   
    loss    lr  ltpb  ltps  ltrunc  ltrunclen   mrr  rank  tpb   tps  
   5.247 5e-09 141.5  1315       0          0 .4123 4.124 1677 15586



({'accuracy': ExactMatchMetric(0.2307),
  'bleu-4': BleuMetric(0.2308),
  'clen': AverageMetric(155.8),
  'ctpb': GlobalAverageMetric(1570),
  'ctps': GlobalTimerMetric(1.425e+04),
  'ctrunc': AverageMetric(0),
  'ctrunclen': AverageMetric(0),
  'exps': GlobalTimerMetric(90.34),
  'exs': SumMetric(7801),
  'f1': F1Metric(0.3182),
  'gpu_mem': GlobalAverageMetric(0.4421),
  'hits@1': AverageMetric(0.2307),
  'hits@10': AverageMetric(1),
  'hits@100': AverageMetric(1),
  'hits@5': AverageMetric(0.7214),
  'llen': AverageMetric(14.35),
  'loss': AverageMetric(4.727),
  'lr': GlobalAverageMetric(5e-09),
  'ltpb': GlobalAverageMetric(142.8),
  'ltps': GlobalTimerMetric(1297),
  'ltrunc': AverageMetric(0),
  'ltrunclen': AverageMetric(0),
  'mrr': AverageMetric(0.4398),
  'rank': AverageMetric(3.95),
  'tpb': GlobalAverageMetric(1713),
  'tps': GlobalTimerMetric(1.555e+04)},
 {'accuracy': ExactMatchMetric(0.1982),
  'bleu-4': BleuMetric(0.1988),
  'clen': AverageMetric(152.5),
  'ctpb': Glob

# Model prediction

In [43]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    datapath = './data',
    task='personachat',
    model_file='/content/from_pretrained/model',
    num_examples=10,
   # skip_generation=False,
)

18:11:27 | Overriding opt["model_file"] to /content/from_pretrained/model (previously: from_pretrained/model)
18:11:27 | Using CUDA
18:11:27 | loading dictionary from /content/from_pretrained/model.dict
18:11:27 | num words = 54944
18:11:32 | Total parameters: 256,081,920 (256,081,920 trainable)
18:11:32 | Loading existing model parameters from /content/from_pretrained/model
18:11:33 | creating task(s): personachat
18:11:33 | loading fbdialog data: ./data/Persona-Chat/personachat/valid_self_original.txt


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


18:11:35 | Opt:
18:11:35 |     activation: gelu
18:11:35 |     adafactor_eps: '[1e-30, 0.001]'
18:11:35 |     adam_eps: 1e-08
18:11:35 |     add_p1_after_newln: False
18:11:35 |     aggregate_micro: False
18:11:35 |     allow_missing_init_opts: False
18:11:35 |     attention_dropout: 0.1
18:11:35 |     batchsize: 256
18:11:35 |     betas: '[0.9, 0.999]'
18:11:35 |     bpe_add_prefix_space: None
18:11:35 |     bpe_debug: False
18:11:35 |     bpe_dropout: None
18:11:35 |     bpe_merge: None
18:11:35 |     bpe_vocab: None
18:11:35 |     candidates: batch
18:11:35 |     cap_num_predictions: 100
18:11:35 |     checkpoint_activations: False
18:11:35 |     codes_attention_num_heads: 4
18:11:35 |     codes_attention_type: basic
18:11:35 |     data_parallel: True
18:11:35 |     datapath: ./data
18:11:35 |     datatype: train
18:11:35 |     delimiter: '\n'
18:11:35 |     dict_class: parlai.core.dict:DictionaryAgent
18:11:35 |     dict_endtoken: __start__
18:11:35 |     dict_file: /content/from_p

In [26]:
!zip -r ./image_seq2seq_personachat.zip ./from_pretrained

  adding: from_pretrained/ (stored 0%)
  adding: from_pretrained/model.valid (deflated 44%)
  adding: from_pretrained/model.trainstats (deflated 81%)
  adding: from_pretrained/model.test (deflated 45%)
  adding: from_pretrained/model (deflated 8%)
  adding: from_pretrained/model.dict.opt (deflated 69%)
  adding: from_pretrained/model.dict (deflated 55%)
  adding: from_pretrained/model.opt (deflated 66%)
  adding: from_pretrained/model.dict.codecs (deflated 64%)


# Model testing

In [46]:
# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # Model files that begin with "zoo:" are special files distributed by the ParlAI team.
    # They'll be automatically downloaded when you ask to use them.
    model_file='pretrained_models/model'
)

18:29:27 | Overriding opt["model_file"] to pretrained_models/model (previously: from_pretrained/model)
18:29:27 | Using CUDA
18:29:27 | loading dictionary from pretrained_models/model.dict
18:29:27 | num words = 54946
18:29:28 | ImageSeq2seq: full interactive mode on.
18:29:30 | Total parameters: 88,559,104 (88,559,104 trainable)
18:29:30 | Loading existing model params from pretrained_models/model
18:29:32 | Opt:
18:29:32 |     activation: gelu
18:29:32 |     adafactor_eps: '[1e-30, 0.001]'
18:29:32 |     adam_eps: 1e-08
18:29:32 |     add_p1_after_newln: False
18:29:32 |     aggregate_micro: False
18:29:32 |     allow_missing_init_opts: False
18:29:32 |     attention_dropout: 0.0
18:29:32 |     batchsize: 16
18:29:32 |     beam_block_full_context: True
18:29:32 |     beam_block_list_filename: None
18:29:32 |     beam_block_ngram: -1
18:29:32 |     beam_context_block_ngram: -1
18:29:32 |     beam_delay: 30
18:29:32 |     beam_length_penalty: 0.65
18:29:32 |     beam_min_length: 1
18:2

In [ ]:
# call it with particular args
Interactive.main(
    # the model_file is a filename path pointing to a particular model dump.
    # Model files that begin with "zoo:" are special files distributed by the ParlAI team.
    # They'll be automatically downloaded when you ask to use them.
    model_file='from_pretrained/model'
)